In [163]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
import scipy.linalg as linalg
import sklearn.linear_model
import sklearn.model_selection
import pandas as pd
from sklearn.datasets.samples_generator import make_blobs
from sklearn.datasets import load_iris
from IPython.display import Markdown as md

# Applied Machine Learning

## Recomenders and embeddings

### Recommender systems

- The main goal: predict the rating (preference, score, ..) that a user $u$ would give to some item $i$
- Important part of any massive online service
- A huge field of study
- Let us consider the classic user-movie setting

### Two approaches

- Model user-item interaction directly (content-based)
- Model user-user interaction to predict items (collaborative filtering)

### Content-based recommenders

- The rating problem is cast to classification
- Collect user actions (mouseovers, clicks, likes, whatever)
- User features: genres user interacted with, countries of movies they did like, etc
- Movie features: category, year, country, etc
- Combine these two sets of features and predict if they match

### Toy example

We have a set of users with counts of each genre assigned:

In [67]:
users = [
    {'comedy': 2, 'action': 1},
    {'documentary': 1, 'action': 1}, 
    {'comedy': 3}
]

Let's vectorize them:

In [68]:
from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer().fit(users)

vectorizer.transform(users).todense()

matrix([[1., 2., 0.],
        [1., 0., 1.],
        [0., 3., 0.]])

In [69]:
genres = vectorizer.feature_names_
genres

['action', 'comedy', 'documentary']

In [115]:
features, labels = [], []
for user in users:
    for genre in genres:
        label = +1 if genre in user else -1
        user_features = np.ravel(vectorizer.transform(user).todense())
        item_features = np.ravel(vectorizer.transform({genre: 1}).todense())
        features.append(np.concatenate((user_features, item_features)))
        labels.append(label)
        
features = np.vstack(features)
labels = np.array(labels)

We've got a feature matrix and a labels vector:

In [116]:
print('Features', features)
print('Labels', labels.T)

Features [[1. 2. 0. 1. 0. 0.]
 [1. 2. 0. 0. 1. 0.]
 [1. 2. 0. 0. 0. 1.]
 [1. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 0. 1.]
 [0. 3. 0. 1. 0. 0.]
 [0. 3. 0. 0. 1. 0.]
 [0. 3. 0. 0. 0. 1.]]
Labels [ 1  1 -1  1 -1  1 -1  1 -1]


You know the next step

In [122]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression().fit(features, labels)

### Simple content-based model

- Our model obviously learns some tautology
- Although it also tends to learn dependencies between genres
- Given good features and good classifier such a model would produce quite good results

In [133]:
lr.predict_proba(np.array([1, 0, 0, 1, 0, 0]).reshape(-1, 1).T)

array([[0.32236604, 0.67763396]])

In [134]:
lr.predict_proba(np.array([1, 0, 0, 0, 0, 1]).reshape(-1, 1).T)

array([[0.45985691, 0.54014309]])

### Pitfalls

- Train set is rather high-maintenance (you need proper positive and negative examples)
- Feature engineering is very important
- Scalability might become a problem
- Can you leverage other users?

### Collaborative filtering (CF)

- Another approach for recommenders leveraging user-user relations
- Could be user-centric or item-centric
- User-centric: given a user, find other like-minded users and use their ratings

### User-item rating matrix

In [161]:
likes = ('Lliane', 'Titanic', 4), ('Roxanne', 'Titanic', 5), ('Roxanne', 'Ghost', 4)
users = {name: idx for (idx, name) in enumerate({x[0] for x in likes})}
items = {name: idx for (idx, name) in enumerate({x[1] for x in likes})}

R = np.zeros((len(users), len(items)))
for user, item, score in likes:
    R[users[user], items[item]] = score
R

array([[5., 4.],
       [4., 0.]])

### Memory-based CF

- The simplest approach to fill in missed ratings is:
$
R(user, item) = \mathop{avg}_{\text{item}~x} R(user, x)
$
- Let's try to clarify using other users:
$R(user, item) = \mathop{avg}_{\text{item}~x} R(user, x) + \mathop{avg}_{\text{other users}~u} \left(R(u, item) - \mathop{avg}_{\text{item}~x} R(u, x)\right)$
- Everybody likes Shawshank's Redemption so should you

### Memory-base CF with similarity

- What if we had a function $S$ to measure similarity between users?
- The scoring rule would become:
$
R(user, item) = \mathop{avg}_{\text{item}~x} R(user, x) + \frac{1}{\text{normalizer}} \sum_{\text{other users}~u} S(user, u) \left(R(u, item) - \mathop{avg}_{\text{item}~x} R(u, x)\right)
$
- What is $S$?

### Pearson similarity

$S(u_1, u_2) = \frac{\sum_{item} \left(R(u_1, item) - \mathop{avg}_{x} R(u_2, x)\right) \left((R(u_2, item) - \mathop{avg}_{x} R(u_2, x)\right)}{\sqrt{\sum_{item} \left(R(u_1, item) - \mathop{avg}_{x} R(u_1, x)\right)^2} \sqrt{\sum_{item}\left(R(u_2, item) - \mathop{avg}_{x} R(u_2, x)\right)^2} }$

- Sums are computed on items rated by both of the users 
- Measures if users $u_1$ and $u_2$ often agree on the high and the low ratings
- Normalized using user's average rating

### Centricity

- We considered a model that measures users similarity (user-centric)
- Everything can be transposed to compute items similarity (item-centric)
- There is no single solution

### Matrix factorization

- A technique from linear algebra
- The matrix $X$ can be factorized into a product of matrices with some properties
- One of the most famous is LU and the other one is SVD

### LU decomposition

Useful to solve linear systems, $X=PLU$

In [181]:
X = np.array([[1, 2], [3, 4]])
P, L, U = linalg.lu(X)
print('P', P)
print('L', L)
print('U', U)
print('PLU', P.dot(L).dot(U))

P [[0. 1.]
 [1. 0.]]
L [[1.         0.        ]
 [0.33333333 1.        ]]
U [[3.         4.        ]
 [0.         0.66666667]]
PLU [[1. 2.]
 [3. 4.]]


### Singular Value Decomposition

Finds structure in matrix you don't see, $X=U\Sigma V^{T}$

In [188]:
U, s, Vh = linalg.svd(X, full_matrices=False)
U.dot(np.diag(s)).dot(Vh)

array([[1., 2.],
       [3., 4.]])

### Low-rank SVD decomposition

$U$, $\Sigma$, $V^{T}$ are of the same size as $X$ but we can cut them and still get something reasonable

In [216]:
np.outer(U[:, 0].dot(s[0]), Vh[0])

array([[1.27357371, 1.80720735],
       [2.87897923, 4.08528566]])

We lost some information but kept something basic

### Low-rank decomposition

- We can find matrices $U, I$ such that $R \sim U I$
- $R$ is of size #users x #items
- $U$ might be of size #users x $d$
- $I$ might be of size #items x $d$
- $d$ is a small number (3 - 10)
- Such a decomposition is not exact but useful, it tries to capture the essential information

### Embeddings

- The $U$ matrix contains a vector for each user
- The $I$ matrix contains a vector for each item
- These vectors are also called embeddings
- The idea to vectorize everything was so hot it is still so

### Embeddings

In [227]:
users = {
    'Lliane': [9.3, 5.4, -3.2],
    'Roxanne': [5.2, -1.2, 5.6],
}
items = {
    'Titanic': [3.7, -0.5, 8.9],
}
score = np.array(users['Lliane']).dot(np.array(items['Titanic']))
score

3.2300000000000004

### Real-world recommenders

- Usually hybrid with a lot of running parts
- Different ways to introduce contextual information
- A lot of nuances to consider: train data, features, etc
- Additional metrics like diversity and serendipity
- Filter bubble and other problems

### PMI matrix

- Pointwise mutual information: $\log \frac{P(u, v)}{P(u) P(v)}$
- Measures how often $u, v$ happen together compared to discretely
- Becomes an interesting tool if $u, v$ are some language's words

### PMI matrix decomposition

- It is long known that PMI matrix could provide useful insights on words
- One obvious thing was to factorize it the same way we did with ratings
- What if we had embeddings for words?

### Word embeddings

Similar to user-items setting:

In [231]:
words = {
    'Moscow': [4.2, 2.4],
    'St. Petersburg': [-3.4, 2.3],
    'New York': [-2.3, -3.1],
    'Washington': [3.3, -2.7],
}
np.array(words['Moscow']).dot(np.array(words['Washington']))

7.379999999999999

### Word embedding

- Factorizing PMI sounds like a promising thing
- Nobody knew how to do that properly and it didn't work well
- The key was to consider PMI between words and "contexts"

### Word2vec

- Introduced in 2013 and started a revolution
- Small program made in totally unreadable C
- Consists of two regimes: skip-gram and Continuous Bag of Words (CBoW)
- Considers word contexts $\dots, w_{i-3}, w_{i-2}, w_{i-1}, \mathbf{w_{i}}, w_{i+1}, w_{i+2}, w_{i+3}, \dots$

### Skip-gram

- For any context, $w_i$ is fixed
- We try to fit $w_{j}$ so that it is similar to $w_i$
- Learn to predict the context given a word

### Continuous Bag of Words

- For any context, everything but $w_i$ is fixed
- We try to fit vector of $w_i$ to the vector of context
- Learn to predict the word given a context

### What do we get?

In [241]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [247]:
model.get_vector('machine')[:10]

array([ 0.25585938, -0.02209473,  0.02905273,  0.05444336, -0.07421875,
        0.35351562, -0.06347656,  0.14453125,  0.07226562,  0.10009766],
      dtype=float32)

### Algebra of vectors

The vectors we learn in word2vec are actually following some algebraic rules

In [251]:
model.most_similar(positive=['berlin', 'russia'], negative=['moscow'])

[('germany', 0.48318061232566833),
 ('korea', 0.4527997374534607),
 ('european', 0.4290318191051483),
 ('asia', 0.42432671785354614),
 ('german', 0.4178942143917084),
 ('enregistrer', 0.41728442907333374),
 ('charles', 0.41599637269973755),
 ('facto', 0.41378772258758545),
 ('vietnam', 0.40711793303489685),
 ('eu', 0.40642309188842773)]

### Learning embeddings

- Once we have the objective (skip-gram or CBoW) it is possible to employ optimization
- Optimization refines the embeddings so that they fit together better
- In the end, you get a lookup table (word -> vector)

### fastText

- Very similar thing to word2vec
- Consider character n-grams 
- This enables capturing some language structure

In [223]:
word = 'character'
[word[i:i+3] for i in range(len(word) - 2)]

['cha', 'har', 'ara', 'rac', 'act', 'cte', 'ter']

### PMI?

- All the methods for finding word embeddings are doing something similar
- It is some equivalent of the PMI matrix factorization
- Not really an important issue for practicioners

### Word embeddings in practice

- One doesn't really need to train word embeddings
- Just google 'pretrained word2vec' or 'pretrained fasttext' and enjoy the results

### Practical considerations

- To embed sentences just sum the word embeddings
- Do not forget to normalize the embeddings
- Each component of embedding is important, hints for neural networks and linear models
- You can actually embed URLs the same way

### Next class

- Non-parametric methods
- k nearest neighbor
- k-means